# Climate Processor Package Testing

This notebook demonstrates how to use the modular climate_processor package to process and visualize climate data.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import xarray as xr
import os

# Import our custom package
from climate_processor import ClimateDataProcessor

## 1. Initialize the Climate Data Processor

First, we'll create an instance of the ClimateDataProcessor class with default parameters.

In [ ]:
# Create a processor instance with default parameters
processor = ClimateDataProcessor(
    data_dir="data/climate_variables",
    lon_slice=slice(187.5, 192.5),
    lat_slice=slice(-12.5, -17.5),
    time_slice=slice("1958-01-01", "2024-12-31"),
    target_lons=np.array([187.5, 190.0, 192.5]),
    target_lats=np.array([-12.5, -15.0, -17.5])
)

## 2. Process Climate Variables

Now we'll demonstrate the different ways to process climate variables using our modular approach.

### 2.1 Process All Variables

In [ ]:
# Process all variables
print("Processing all climate variables...")
processor.process_all_variables()

# Validate grid consistency
processor.validate_grid_consistency()

### 2.2 Process Variables by Name

In [ ]:
# Create a new processor for demonstration
processor_by_name = ClimateDataProcessor()

# Process specific variables by name
success1 = processor_by_name.process_variable("air_2m")
success2 = processor_by_name.process_variable("skin_temp")

# Display the keys of the climate data dictionary
print("Processed variables:")
print(list(processor_by_name.climate_data.keys()))

### 2.3 Process Variables by Description

In [ ]:
# Create a new processor for demonstration
processor_by_desc = ClimateDataProcessor()

# Process variables by description
success1 = processor_by_desc.process_by_description("2m air temperature")
success2 = processor_by_desc.process_by_description("sea level pressure")

# Display the keys of the climate data dictionary
print("Processed variables:")
print(list(processor_by_desc.climate_data.keys()))

## 3. Examine the Processed Data

Let's look at the keys of our climate data dictionary and examine a few variables.

In [ ]:
# Display the keys of the climate data dictionary
print("All variables in the climate_data dictionary:")
print(list(processor.climate_data.keys()))

In [ ]:
# Examine the air_2m variable if it exists
if 'air_2m' in processor.climate_data:
    processor.climate_data['air_2m']

In [ ]:
# Examine the skin_temp variable if it exists
if 'skin_temp' in processor.climate_data:
    processor.climate_data['skin_temp']

## 4. Save Processed Data to NetCDF

In [ ]:
# Save the processed data to a NetCDF file
output_path = "AS_climate_var_ds.nc"
processor.save_to_netcdf(output_path)
print(f"Processed climate data saved to {output_path}")

## 5. Visualizations

Now let's create some visualizations of our climate data.

### 5.1 Visualize 2m Air Temperature and Skin Temperature

In [ ]:
# Check if the required variables exist
if 'air_2m' in processor.climate_data and 'skin_temp' in processor.climate_data:
    # Extract the first time step for both variables
    air_2m_first = processor.climate_data['air_2m'].isel(time=0)
    skin_temp_first = processor.climate_data['skin_temp'].isel(time=0)
    
    # Squeeze any extra dimensions to obtain 2D arrays
    data_2d_air = np.squeeze(air_2m_first.values)
    data_2d_skin = np.squeeze(skin_temp_first.values)
    
    # Define the desired target extent explicitly, subtracting 360 from longitudes
    extent_target = [187.5-360, 192.5-360, -17.5, -12.5]
    
    # Create a figure with two subplots side by side using PlateCarree projection
    fig, (ax1, ax2) = plt.subplots(
        ncols=2, 
        figsize=(12, 6), 
        subplot_kw={'projection': ccrs.PlateCarree()}
    )
    
    # Plot 1: 2m Air Temperature using imshow
    cmap_air = plt.get_cmap('YlGnBu_r', 13)
    im1 = ax1.imshow(data_2d_air,
                     cmap=cmap_air,
                     transform=ccrs.PlateCarree(),
                     extent=extent_target)
    ax1.coastlines(resolution='10m', linewidth=0.65)
    ax1.set_title('2m Air Temperature — First Time Step (Raw)')
    fig.colorbar(im1, ax=ax1, orientation='vertical', label='2m Air Temp (K)')
    
    # Plot 2: Skin Temperature using imshow
    cmap_skin = plt.get_cmap('YlGnBu_r', 13)
    im2 = ax2.imshow(data_2d_skin,
                     cmap=cmap_skin,
                     transform=ccrs.PlateCarree(),
                     extent=extent_target)
    ax2.coastlines(resolution='10m', linewidth=0.65)
    ax2.set_title('Skin Temperature — First Time Step (Raw)')
    fig.colorbar(im2, ax=ax2, orientation='vertical', label='Skin Temp (C)')
    
    plt.tight_layout()
    plt.show()
else:
    print("Required variables not found in climate_data dictionary")

### 5.2 Visualize Geopotential Height at 500 hPa

In [ ]:
# Check if the required variable exists
if 'hgt_500' in processor.climate_data:
    # Extract the first time step for geopotential height at 500 hPa
    hgt_500_first = processor.climate_data['hgt_500'].isel(time=0)
    data_2d_hgt = np.squeeze(hgt_500_first.values)
    
    # Create a figure with PlateCarree projection
    fig, ax = plt.subplots(
        figsize=(8, 6), 
        subplot_kw={'projection': ccrs.PlateCarree()}
    )
    
    # Plot geopotential height using imshow
    cmap_hgt = plt.get_cmap('viridis', 13)
    im = ax.imshow(data_2d_hgt,
                   cmap=cmap_hgt,
                   transform=ccrs.PlateCarree(),
                   extent=extent_target)
    ax.coastlines(resolution='10m', linewidth=0.65)
    ax.set_title('Geopotential Height at 500 hPa — First Time Step')
    fig.colorbar(im, ax=ax, orientation='vertical', label='Geopotential Height (m)')
    
    plt.tight_layout()
    plt.show()
else:
    print("hgt_500 variable not found in climate_data dictionary")

### 5.3 Time Series Visualization

In [ ]:
# Check if the required variables exist
if 'air_2m' in processor.climate_data and 'skin_temp' in processor.climate_data:
    # Extract time series for a specific grid point (center point)
    try:
        air_2m_center = processor.climate_data['air_2m'].sel(lat=-15.0, lon=190.0, method='nearest')
        skin_temp_center = processor.climate_data['skin_temp'].sel(lat=-15.0, lon=190.0, method='nearest')
        
        # Create a figure with two subplots stacked vertically
        fig, (ax1, ax2) = plt.subplots(nrows=2, figsize=(12, 8))
        
        # Plot 1: 2m Air Temperature time series
        ax1.plot(air_2m_center.time, air_2m_center.values, 'b-', label='2m Air Temperature')
        ax1.set_title('2m Air Temperature Time Series at Center Point (15°S, 190°E)')
        ax1.set_ylabel('Temperature (K)')
        ax1.grid(True)
        
        # Plot 2: Skin Temperature time series
        ax2.plot(skin_temp_center.time, skin_temp_center.values, 'r-', label='Skin Temperature')
        ax2.set_title('Skin Temperature Time Series at Center Point (15°S, 190°E)')
        ax2.set_ylabel('Temperature (°C)')
        ax2.set_xlabel('Time')
        ax2.grid(True)
        
        plt.tight_layout()
        plt.show()
    except Exception as e:
        print(f"Error creating time series plot: {e}")
else:
    print("Required variables not found in climate_data dictionary")

## 6. Demonstrating the Modular Structure

Let's demonstrate how the modular structure makes it easy to extend the functionality.

### 6.1 Adding a New Variable Configuration

In [ ]:
# Create a new processor
custom_processor = ClimateDataProcessor()

# Add a new variable configuration
custom_processor.variable_configs["custom_temp_diff"] = {
    "description": "Custom temperature difference between 1000 and 850 hPa",
    "variable": "Air",
    "levels": [1000, 850],
    "operation": "diff"
}

# Process the new variable
success = custom_processor.process_variable("custom_temp_diff")

# Check if it was processed successfully
if success and "custom_temp_diff" in custom_processor.climate_data:
    print("Successfully processed custom variable!")
    print(custom_processor.climate_data["custom_temp_diff"])
else:
    print("Failed to process custom variable.")

### 6.2 Processing Variables with Different Time Intervals

In [ ]:
# Create a new processor
interval_processor = ClimateDataProcessor()

# Add a variable configuration with a different time interval
interval_processor.variable_configs["daily_air_2m"] = {
    "description": "Daily 2m air temperature",
    "variable": "Air 2m",
    "time_interval": "daily",  # Override the default monthly
    "interpolate": True
}

# Try to process the variable
success = interval_processor.process_variable("daily_air_2m")

# Check if it was processed successfully
if success and "daily_air_2m" in interval_processor.climate_data:
    print("Successfully processed daily air temperature!")
    print(interval_processor.climate_data["daily_air_2m"])
else:
    print("Failed to process daily air temperature.")
    print("This is expected if the daily data file doesn't exist.")

## 7. Conclusion

In this notebook, we've demonstrated how to use the modular climate_processor package to process climate data and create visualizations. The modular design makes it easy to work with climate variables and extend the functionality as needed.